In [ ]:
!pip install q scipy==1.1.0
#use an older version of scipy for executing sparse matrix operations

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
valset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
valloader = torch.utils.data.DataLoader(valset, batch_size = 64, shuffle = True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)



In [ ]:
inputsize = 784
hiddensizes = [128,128,128]
outputsize = 10
net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)
w_init= list(net.parameters())
w_init[0]

Parameter containing:
tensor([[ 7.8869e-02,  5.6885e-02,  2.2825e-02,  ..., -5.7455e-02,
          7.3772e-02,  4.3002e-02],
        [-7.9931e-02,  1.8043e-03, -6.8392e-02,  ..., -1.2933e-02,
         -1.7669e-02, -8.0947e-02],
        [-7.1699e-02,  3.2029e-02, -3.1321e-03,  ..., -3.5651e-02,
         -2.7409e-02,  5.7684e-02],
        ...,
        [-5.6637e-02, -4.1404e-02,  5.7319e-02,  ..., -3.0192e-02,
         -6.6982e-02,  3.2356e-02],
        [ 4.2089e-02,  1.2743e-02,  2.1949e-05,  ..., -8.0196e-03,
         -6.3435e-02, -7.9036e-03],
        [-3.0654e-02,  7.1574e-02,  4.4725e-02,  ...,  5.1446e-02,
          2.7728e-02, -7.5360e-02]], requires_grad=True)

In [ ]:
#define the model 
#set random seed for reproducibility
torch.manual_seed(0)
inputsize = 784
hiddensizes = [128,128,128]
outputsize = 10
net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)
w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')
print(net)

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)
logps = (net(images))
loss = criterion(logps,labels)

optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()
epochs =5
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
  (7): LogSoftmax()
)
Epoch 0 - Training loss: 0.454596993582907

 Training time in minutes =  0.1601380427678426
Epoch 1 - Training loss: 0.21611409411549187

 Training time in minutes =  0.3144444465637207
Epoch 2 - Training loss: 0.1602034371223118

 Training time in minutes =  0.4776981115341187
Epoch 3 - Training loss: 0.12960973877245302

 Training time in minutes =  0.6433154026667277
Epoch 4 - Training loss: 0.11076642346900029

 Training time in minutes =  0.8025577108065287


In [ ]:
device(net)

In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in valloader:
    for i in range(len(labels)):
        img = images[i].view(1, 784)
        with torch.no_grad():
            logps = net(img)

    
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9639


In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*784))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
dim_w

134400

In [ ]:
#A run  1
from scipy import sparse
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,134400])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0916

 Dist from Wi = 0.025832757149639087 , Dist from Wf =6.40098835195938

 n_rows = 4 , Accuracy =0.0917

 Dist from Wi = 0.03684976624590671 , Dist from Wf =6.400934408973685

 n_rows = 8 , Accuracy =0.0916

 Dist from Wi = 0.040891594585081674 , Dist from Wf =6.400909864289846

 n_rows = 16 , Accuracy =0.0922

 Dist from Wi = 0.07667619678527925 , Dist from Wf =6.4005812215846944

 n_rows = 32 , Accuracy =0.092

 Dist from Wi = 0.09529803783627736 , Dist from Wf =6.400331045910312

 n_rows = 64 , Accuracy =0.0922

 Dist from Wi = 0.14052136505122295 , Dist from Wf =6.399497867741352

 n_rows = 128 , Accuracy =0.0926

 Dist from Wi = 0.19704959357418217 , Dist from Wf =6.398006773275962

 n_rows = 256 , Accuracy =0.096

 Dist from Wi = 0.2805389469042387 , Dist from Wf =6.394889921844501

 n_rows = 512 , Accuracy =0.0979

 Dist from Wi = 0.40489712607592526 , Dist from Wf =6.388221781554407

 n_rows = 1024 , Accuracy =0.1059

 Di

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#winit, AI run 1
I = sparse.identity(dim_w)
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,134400])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs ) + I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:] 
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0917

 Dist from Wi = 0.014612138284833563 , Dist from Wf =6.401023800821358

 n_rows = 4 , Accuracy =0.0914

 Dist from Wi = 0.035277786157421165 , Dist from Wf =6.400943265727401

 n_rows = 8 , Accuracy =0.0916

 Dist from Wi = 0.06186766666165217 , Dist from Wf =6.400741488693242

 n_rows = 16 , Accuracy =0.0912

 Dist from Wi = 0.06687085516790518 , Dist from Wf =6.400691173769715

 n_rows = 32 , Accuracy =0.09

 Dist from Wi = 0.1015456340818575 , Dist from Wf =6.400234972057618

 n_rows = 64 , Accuracy =0.0901

 Dist from Wi = 0.13501070766692214 , Dist from Wf =6.399616498051188

 n_rows = 128 , Accuracy =0.0935

 Dist from Wi = 0.20310296592941965 , Dist from Wf =6.39781747147375

 n_rows = 256 , Accuracy =0.0972

 Dist from Wi = 0.3383802298575421 , Dist from Wf =6.392090271072322

 n_rows = 512 , Accuracy =0.102

 Dist from Wi = 0.47921013644179544 , Dist from Wf =6.3830773815433846

 n_rows = 1024 , Accuracy =0.1067

 Dist

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#I run 1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,784*128,784*128+128*128,784*128+128*128+128*128,784*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:]

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0916

 Dist from Wi = 0.0025935853493631094 , Dist from Wf =6.401039953521266

 n_rows = 4 , Accuracy =0.0915

 Dist from Wi = 0.0036679245540393586 , Dist from Wf =6.40103942806088

 n_rows = 8 , Accuracy =0.0915

 Dist from Wi = 0.005187220747449995 , Dist from Wf =6.401038377169865

 n_rows = 16 , Accuracy =0.0909

 Dist from Wi = 0.0072451247687326265 , Dist from Wf =6.401036378698967

 n_rows = 32 , Accuracy =0.0903

 Dist from Wi = 0.010310484417144281 , Dist from Wf =6.4010321751395365

 n_rows = 64 , Accuracy =0.0889

 Dist from Wi = 0.014423851323990254 , Dist from Wf =6.40102422786959

 n_rows = 128 , Accuracy =0.0893

 Dist from Wi = 0.14437136875126969 , Dist from Wf =6.399412169968847

 n_rows = 256 , Accuracy =0.1087

 Dist from Wi = 0.501429982009391 , Dist from Wf =6.381370321992179

 n_rows = 512 , Accuracy =0.1127

 Dist from Wi = 0.7717346311182466 , Dist from Wf =6.354348501018283

 n_rows = 1024 , Accuracy =0.114

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
# A=Irev run number 3  
from scipy import sparse

n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,784*128,784*128+128*128,784*128+128*128+128*128,784*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0916

 Dist from Wi = 0.10228690160097022 , Dist from Wf =6.400223168220103

 n_rows = 4 , Accuracy =0.0916

 Dist from Wi = 0.1148765831933198 , Dist from Wf =6.4000095768595795

 n_rows = 8 , Accuracy =0.0916

 Dist from Wi = 0.1316354506385036 , Dist from Wf =6.399686814321128

 n_rows = 16 , Accuracy =0.0916

 Dist from Wi = 0.29064334831246663 , Dist from Wf =6.394438650682499

 n_rows = 32 , Accuracy =0.0908

 Dist from Wi = 0.43393762728252994 , Dist from Wf =6.386314848869332

 n_rows = 64 , Accuracy =0.0921

 Dist from Wi = 0.5835393940382239 , Dist from Wf =6.374386322530665

 n_rows = 128 , Accuracy =0.0903

 Dist from Wi = 0.7576553987839039 , Dist from Wf =6.356042598185926

 n_rows = 256 , Accuracy =0.0904

 Dist from Wi = 1.0445383082904705 , Dist from Wf =6.3152402120405675

 n_rows = 512 , Accuracy =0.1032

 Dist from Wi = 1.403636588591378 , Dist from Wf =6.245248076772296

 n_rows = 1024 , Accuracy =0.1545

 Dist f

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
#I and UI section to be added to the report MNIST128here

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
valset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
valloader = torch.utils.data.DataLoader(valset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


#define the model 
inputsize = 784
hiddensizes = [128,128,128]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


In [ ]:
#Rerun for randomizing w_init

net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
epochs =15
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Epoch 0 - Training loss: 0.435463518444409

 Training time in minutes =  0.15306503375371297
Epoch 1 - Training loss: 0.21181978328999426

 Training time in minutes =  0.30463809967041017
Epoch 2 - Training loss: 0.1616667501334506

 Training time in minutes =  0.45702494780222574
Epoch 3 - Training loss: 0.1318252895102858

 Training time in minutes =  0.6036313454310099
Epoch 4 - Training loss: 0.11256090790204117

 Training time in minutes =  0.7596362471580506
Epoch 5 - Training loss: 0.0970835502404394

 Training time in minutes =  0.9081552624702454
Epoch 6 - Training loss: 0.08550892844955042

 Training time in minutes =  1.0666768431663514
Epoch 7 - Training loss: 0.07560704718095693

 Training time in minutes =  1.2263810873031615
Epoch 8 - Training loss: 0.06973307509087463

 Training time in minutes =  1.3808390935262045
Epoch 9 - Training loss: 0.060990888451629165

 Training time in minutes =  1.5300189852714539
Epoch 10 - Training loss: 0.05444923504047207

 Training time

In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*784))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
dim_w

134400

In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in valloader:
    for i in range(len(labels)):
        img = images[i].view(1, 784)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
np.save('model_accuracy3.npy',(correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9758


In [ ]:
#I run 1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,784*128,784*128+128*128,784*128+128*128+128*128,784*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:]

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1049

 Dist from Wi = 0.0047616034309549615 , Dist from Wf =9.156715864838445

 n_rows = 4 , Accuracy =0.1049

 Dist from Wi = 0.006734024734172078 , Dist from Wf =9.15671462671859

 n_rows = 8 , Accuracy =0.105

 Dist from Wi = 0.00952350008190671 , Dist from Wf =9.156712150395332

 n_rows = 16 , Accuracy =0.1046

 Dist from Wi = 0.013529444962984347 , Dist from Wf =9.156707107709098

 n_rows = 32 , Accuracy =0.104

 Dist from Wi = 0.019090258242915714 , Dist from Wf =9.156697202828733

 n_rows = 64 , Accuracy =0.1039

 Dist from Wi = 0.03067321538216925 , Dist from Wf =9.15666572809718

 n_rows = 128 , Accuracy =0.1046

 Dist from Wi = 0.12368738234605282 , Dist from Wf =9.155881690678385

 n_rows = 256 , Accuracy =0.1046

 Dist from Wi = 0.2716220022124977 , Dist from Wf =9.152687561048142

 n_rows = 512 , Accuracy =0.1072

 Dist from Wi = 0.5293863978412934 , Dist from Wf =9.141401322774968

 n_rows = 1024 , Accuracy =0.1046

 Dist from Wi = 0.75689997671

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
# A=Irev run number 3  
from scipy import sparse

n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,784*128,784*128+128*128,784*128+128*128+128*128,784*128+128*128+128*128+1280])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1049

 Dist from Wi = 0.029207059017898943 , Dist from Wf =9.15667052208142

 n_rows = 4 , Accuracy =0.1075

 Dist from Wi = 0.15615469081202898 , Dist from Wf =9.155385508802386

 n_rows = 8 , Accuracy =0.1067

 Dist from Wi = 0.1659847597156032 , Dist from Wf =9.155212567810398

 n_rows = 16 , Accuracy =0.1114

 Dist from Wi = 0.33155353795494297 , Dist from Wf =9.150712559889579

 n_rows = 32 , Accuracy =0.1014

 Dist from Wi = 0.5068886889927814 , Dist from Wf =9.142676411162242

 n_rows = 64 , Accuracy =0.0997

 Dist from Wi = 0.7189085260401408 , Dist from Wf =9.128452148827758

 n_rows = 128 , Accuracy =0.0997

 Dist from Wi = 1.1126750108440238 , Dist from Wf =9.088862548333239

 n_rows = 256 , Accuracy =0.1046

 Dist from Wi = 1.5009246103144818 , Dist from Wf =9.032867397255627

 n_rows = 512 , Accuracy =0.1089

 Dist from Wi = 2.002033035897218 , Dist from Wf =8.935173855355446

 n_rows = 1024 , Accuracy =0.1047

 Dist from Wi = 2.8855426339242727 

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)

In [ ]:
#I UI for cifar 192

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

In [ ]:
cuda = torch.device('cuda')     # Default CUDA device
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


#define the model 
inputsize = 3*32*32
hiddensizes = [192,192,192]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


In [ ]:
#Rerun for randomizing w_init

net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()

Sequential(
  (0): Linear(in_features=3072, out_features=192, bias=True)
  (1): ReLU()
  (2): Linear(in_features=192, out_features=192, bias=True)
  (3): ReLU()
  (4): Linear(in_features=192, out_features=192, bias=True)
  (5): ReLU()
  (6): Linear(in_features=192, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
epochs =15
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)

        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Epoch 0 - Training loss: 1.7301762326599082

 Training time in minutes =  0.1356374979019165
Epoch 1 - Training loss: 1.4857225832731829

 Training time in minutes =  0.2702988942464193
Epoch 2 - Training loss: 1.3768593379298744

 Training time in minutes =  0.40099146366119387
Epoch 3 - Training loss: 1.2944727700842007

 Training time in minutes =  0.5364514191945394
Epoch 4 - Training loss: 1.2252646199882489

 Training time in minutes =  0.6687025944391887
Epoch 5 - Training loss: 1.1694155118197127

 Training time in minutes =  0.8046006441116333
Epoch 6 - Training loss: 1.1147881231801895

 Training time in minutes =  0.9402898788452149
Epoch 7 - Training loss: 1.0640769529982905

 Training time in minutes =  1.0735091368357341
Epoch 8 - Training loss: 1.0209258730759097

 Training time in minutes =  1.2061952193578085
Epoch 9 - Training loss: 0.972156114056897

 Training time in minutes =  1.3357619444529216
Epoch 10 - Training loss: 0.9252163156524034

 Training time in minute

In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1, 3072)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
np.save('model_accuracy_5.npy',(correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.5337


In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*inputsize))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
dim_w

665472

In [ ]:
131072*4

524288

In [ ]:
#w_init ,I run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524288,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.111

 Dist from Wi = 0.014752055747059985 , Dist from Wf =14.98614027865965

 n_rows = 4 , Accuracy =0.111

 Dist from Wi = 0.015511568699616817 , Dist from Wf =14.986139511763875

 n_rows = 8 , Accuracy =0.111

 Dist from Wi = 0.030715711109030642 , Dist from Wf =14.986116061872236

 n_rows = 16 , Accuracy =0.111

 Dist from Wi = 0.036546823044637314 , Dist from Wf =14.98610297590869

 n_rows = 32 , Accuracy =0.111

 Dist from Wi = 0.05627827616530191 , Dist from Wf =14.986041866698685

 n_rows = 64 , Accuracy =0.111

 Dist from Wi = 0.0770530421684065 , Dist from Wf =14.985949449517923

 n_rows = 128 , Accuracy =0.111

 Dist from Wi = 0.113944783570084 , Dist from Wf =14.985714352713089

 n_rows = 256 , Accuracy =0.111

 Dist from Wi = 0.1844307031199973 , Dist from Wf =14.985012625639012

 n_rows = 512 , Accuracy =0.1117

 Dist from Wi = 0.3931761496516771 , Dist from Wf =14.980988972366243

 n_rows = 1024 , Accuracy =0.1116

 Dist from Wi = 0.488306390051

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#w_init ,Irev run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524288,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.111

 Dist from Wi = 0.15277687604415321 , Dist from Wf =14.985368774273129

 n_rows = 4 , Accuracy =0.111

 Dist from Wi = 0.21155790818553227 , Dist from Wf =14.984654194418304

 n_rows = 8 , Accuracy =0.111

 Dist from Wi = 0.253756875906661 , Dist from Wf =14.98399898300664

 n_rows = 16 , Accuracy =0.111

 Dist from Wi = 0.37760930717716795 , Dist from Wf =14.981389431089065

 n_rows = 32 , Accuracy =0.111

 Dist from Wi = 0.5656301133623804 , Dist from Wf =14.975469296475206

 n_rows = 64 , Accuracy =0.111

 Dist from Wi = 0.6747281956809028 , Dist from Wf =14.97095053551267

 n_rows = 128 , Accuracy =0.111

 Dist from Wi = 0.9341397526912444 , Dist from Wf =14.95700508114027

 n_rows = 256 , Accuracy =0.1113

 Dist from Wi = 1.3104465901213587 , Dist from Wf =14.928742338497683

 n_rows = 512 , Accuracy =0.1117

 Dist from Wi = 1.9178118809078402 , Dist from Wf =14.862927560351718

 n_rows = 1024 , Accuracy =0.1055

 Dist from Wi = 2.6612565749496966 ,

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)